<div dir="rtl" style="text-align: justify;">

# مشروع: الدليل التقني الذكي (Advanced Tech Guide RAG)
**إعداد المهندس:** لؤي الرازي


---

### 🎯 نظرة عامة (Project Overview)
هذا المشروع هو نظام **Retrieval-Augmented Generation (RAG)** متكامل يعمل محلياً بالكامل (Offline). يهدف النظام إلى تقديم إجابات دقيقة وموثوقة حول لغات البرمجة وإطارات العمل التقنية بناءً على قاعدة معرفية مخصصة، مما يحد من ظاهرة "الهلوسة" في النماذج اللغوية الكبيرة.

### 🛠 التقنيات المستخدمة (Tech Stack)
* **المحرك اللغوي:** `Llama 3.2 (1b)` عبر منصة `Ollama`.
* **قاعدة البيانات المتجهة:** `ChromaDB` لإدارة وفهرسة البيانات دلالياً.
* **تحويل النصوص (Embeddings):** موديل `all-MiniLM-L6-v2` من مكتبة `Sentence-Transformers`.
* **واجهة المستخدم:** `Gradio` لبناء تجربة دردشة تفاعلية.
* **المعالجة البرمجية:** `Python`, `Pandas`, و `NumPy`.

### 🚀 المميزات التقنية
1. **الخصوصية التامة:** معالجة البيانات تتم محلياً دون الحاجة لاتصال خارجي بالإنترنت.
2. **هندسة الأوامر (Prompt Engineering):** استخدام قوالب أوامر صارمة لضمان التزام الموديل بالبيانات المتوفرة فقط.
3. **البحث الدلالي:** القدرة على فهم سياق السؤال بدلاً من الاعتماد على الكلمات المفتاحية التقليدية.

---
</div>

<div dir="rtl">

###  المكتبات والإعدادات الأساسية
</div>

In [1]:
import os
import pandas as pd
import numpy as np
import chromadb
import requests
import gradio as gr
from sentence_transformers import SentenceTransformer

# 1. إعداد المسارات: إنشاء مجلد data لحفظ ملفات المشروع
if not os.path.exists('data'):
    os.makedirs('data')
    print("✅ تم إنشاء مجلد 'data'")

# 2. تحميل الموديل: نستخدم نموذج all-MiniLM-L6-v2 لتحويل النصوص إلى متجهات (384 بُعداً)
# هذا الموديل مثالي للعمل المحلي لخفته وسرعته
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# 3. إعداد قاعدة البيانات: نستخدم PersistentClient لضمان حفظ البيانات في الذاكرة الدائمة (Disk)
# المسار المختار سيحفظ قاعدة البيانات داخل مجلد data
db_path = "data/chroma_db_storage"
client = chromadb.PersistentClient(path=db_path)

print("🚀 البيئة البرمجية والنماذج جاهزة الآن.")

c:\Users\Loay_Alrazi\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):



🚀 البيئة البرمجية والنماذج جاهزة الآن.


<div dir="rtl">

### إعداد وحفظ قاعدة البيانات (CSV)
</div>

In [2]:
# تعريف القاموس التقني (20 لغة و 10 إطارات عمل)
tech_data = {
    "Python": "لغة برمجة عالية المستوى، سهلة التعلم، وتستخدم بكثرة في الذكاء الاصطناعي وRAG.",
    "JavaScript": "لغة الويب الأساسية، تستخدم في تطوير الواجهات الأمامية والخلفية عبر Node.js.",
    "Java": "لغة قوية ومستقرة، تستخدم في تطبيقات الأندرويد والأنظمة المؤسسية الضخمة.",
    "C++": "لغة ذات أداء عالٍ جداً، تستخدم في بناء الألعاب ونظم التشغيل والمتصفحات.",
    "TypeScript": "نسخة مطورة من JavaScript تضيف أنواع البيانات (Static Typing) لتقليل الأخطاء.",
    "Go": "لغة طورتها جوجل، تتميز بالسرعة الفائقة في التعامل مع الشبكات والخوادم.",
    "Rust": "لغة تركز على أمان الذاكرة والأداء العالي، وهي منافس قوي للغة C++.",
    "Swift": "اللغة الرسمية لتطوير تطبيقات Apple (iOS و macOS).",
    "PHP": "لغة عريقة في تطوير المواقع، وتدير نسبة كبيرة من الويب مثل WordPress.",
    "Ruby": "لغة تركز على بساطة الإنتاجية، وتشتهر بإطار عمل Ruby on Rails.",
    "C#": "لغة من تطوير مايكروسوفت، تستخدم في تطوير تطبيقات Windows وألعاب Unity.",
    "SQL": "اللغة القياسية للتعامل مع قواعد البيانات العلائقية (Relational Databases).",
    "Kotlin": "اللغة الحديثة الموصى بها من جوجل لتطوير تطبيقات الأندرويد.",
    "Dart": "لغة من تطوير جوجل، تستخدم لبناء تطبيقات الموبايل عبر إطار عمل Flutter.",
    "R": "لغة متخصصة في الإحصاء وتحليل البيانات والرسوم البيانية العلمية.",
    "MATLAB": "بيئة برمجية للحسابات التقنية والنمذجة الرياضية والهندسية.",
    "Scala": "تجمع بين البرمجة الكائنية والوظيفية، وتستخدم في معالجة البيانات الضخمة.",
    "Perl": "لغة قوية في معالجة النصوص وإدارة الأنظمة (System Administration).",
    "Haskell": "لغة وظيفية (Functional) نقية، تستخدم في الأبحاث والأنظمة المالية المعقدة.",
    "Lua": "لغة خفيفة الوزن، تستخدم غالباً كمحرك سكربت داخل الألعاب.",
    "React": "إطار عمل JavaScript لبناء واجهات المستخدم، مدعوم من شركة Meta.",
    "Angular": "إطار عمل ضخم من جوجل لبناء تطبيقات الويب المعقدة.",
    "Vue.js": "إطار عمل مرن وسهل التعلم لبناء واجهات المواقع التفاعلية.",
    "Django": "إطار عمل Python قوي جداً لتطوير المواقع بسرعة وبأمان عالٍ.",
    "Flask": "إطار عمل Python مصغر (Micro) يتميز بالبساطة والمرونة المطلقة.",
    "Flutter": "إطار عمل لبناء تطبيقات (iOS و Android) من كود واحد باستخدام Dart.",
    "Spring Boot": "إطار عمل Java لتسهيل بناء تطبيقات الويب والخدمات المصغرة (Microservices).",
    "Laravel": "إطار عمل PHP يشتهر بجمالية الكود وسهولة الاستخدام للمطورين.",
    "Express.js": "إطار عمل بسيط وسريع لبناء خوادم الويب باستخدام Node.js.",
    "TensorFlow": "إطار عمل من جوجل لبناء وتدريب نماذج الذكاء الاصطناعي والتعلم العميق."
}

# تحويل القاموس إلى DataFrame وحفظه في مسار محدد
df_to_save = pd.DataFrame([{"tech_name": k, "content": v} for k, v in tech_data.items()])
file_path = 'data/tech_data.csv'
df_to_save.to_csv(file_path, index=False, encoding='utf-8-sig')

print(f"✅ تم حفظ البيانات في: {file_path}")

✅ تم حفظ البيانات في: data/tech_data.csv


<div dir="rtl">

###   بناء الفهرس المتجه (Processing & Ingestion)
</div>

<div dir="rtl">

في هذه الخلية، نقوم بتحميل الملف، تحويل نصوصه إلى أرقام (Embeddings)، وتخزينها في ChromaDB.
</div>

In [3]:
# حذف الكولكشن القديم لضمان عدم تداخل البيانات
try:
    client.delete_collection(name="tech_guide_v1")
except:
    pass

def prepare_database(file_path):
    # 1. قراءة البيانات
    df = pd.read_csv(file_path)
    df['text_id'] = range(1, len(df) + 1)
    
    print(f"جاري معالجة {len(df)} سجل تقني...")

    # 2. توليد المتجهات (Embeddings)
    # نقوم بتحويل كل وصف تقني إلى متجه رقمي يمثل معناه
    df['embeddings'] = df['content'].apply(lambda x: embed_model.encode(x))
    
    # 3. إعداد وتعبئة ChromaDB
    collection = client.get_or_create_collection(name="tech_guide_v1")
    
    collection.add(
        documents=df['content'].tolist(),
        embeddings=df['embeddings'].tolist(),
        ids=[str(i) for i in df['text_id'].tolist()],
        metadatas=[{"name": n} for n in df['tech_name'].tolist()]
    )
    return collection

# تنفيذ العملية وتجهيز المحرك
collection = prepare_database('data/tech_data.csv')
print("✅ قاعدة البيانات المتجهة جاهزة للبحث دلالياً.")

جاري معالجة 30 سجل تقني...
✅ قاعدة البيانات المتجهة جاهزة للبحث دلالياً.


<div dir="rtl">

###  محرك التوليد الصارم (The RAG Engine)
</div>

In [4]:
# هنا تقع دالة الذكاء التي تسترجع المعلومة وتطلب من Llama 3 صياغة الرد بالعربية

def generate_strict_answer(user_query, history):
    try:
        # 1. استرجاع النتائج مع الميتا-داتا (التي تحتوي على اسم التقنية)
        results = collection.query(query_texts=[user_query], n_results=5)
        
        # 2. دمج الاسم مع الوصف لبناء سياق كامل ومفهوم للموديل
        context_list = []
        for doc, meta in zip(results['documents'][0], results['metadatas'][0]):
            context_list.append(f"التقنية: {meta['name']} - الوصف: {doc}")
        
        full_context = " \n".join(context_list)
        
        # سطر الطباعة للتأكد من دمج الأسماء مع الأوصاف
        print(f"DEBUG - السياق المدمج:\n{full_context}")

        # 3. تحديث الـ Prompt ليكون أكثر ذكاءً في الربط
        prompt = f"""
        استخدم السياق الموفر فقط للإجابة على السؤال باللغة العربية. 
        كل سجل في السياق يبدأ بـ "التقنية:" متبوعاً باسمها ثم "الوصف:".
        
        السياق:
        {full_context}
        
        السؤال: {user_query}
        الإجابة:
        """

        response = requests.post('http://localhost:11434/api/generate', 
                                 json={'model': 'llama3', 'prompt': prompt, 'stream': False})
  
        if response.status_code == 200:
            return response.json()['response']
        else:
            return "خطأ في الاتصال بالموديل."
            
    except Exception as e:
        return f"حدث خطأ فني: {str(e)}"

<div dir="rtl">

###  واجهة المستخدم التفاعلية (Gradio)
</div>

In [5]:
# تعريف التنسيق الجمالي ودعم اللغة العربية
custom_css = """
.gradio-container { direction: rtl !important; text-align: right !important; }
.message-wrap { direction: rtl !important; text-align: right !important; }
textarea { direction: rtl !important; text-align: right !important; }
"""

with gr.Blocks(css=custom_css, title="🚀 الدليل التقني الذكي") as demo:
    gr.Markdown("""
    # <div style='text-align: center;'>🚀 الدليل التقني الذكي - نظام RAG</div>
    <div style='text-align: center;'>اسألني عن لغات البرمجة وإطارات العمل وسأجيبك بدقة من قاعدة بياناتي الخاصة.</div>
    """)
    
    # واجهة الدردشة المدمجة
    chat = gr.ChatInterface(
        fn=generate_strict_answer,
        examples=["ما هي مميزات لغة Python؟", "شرح إطار عمل Django", "ما هي لغة Go؟"]
    )

print("جاري تشغيل واجهة المستخدم... يمكنك الآن فتح الرابط أدناه.")
# share=True  رابط  لمشاركته مع الآخرين
demo.launch(share=True)

C:\Users\Loay_Alrazi\AppData\Local\Temp\ipykernel_32664\2576804259.py:8: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=custom_css, title="🚀 الدليل التقني الذكي") as demo:


جاري تشغيل واجهة المستخدم... يمكنك الآن فتح الرابط أدناه.
* Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2026/01/15 17:55:41 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: connectex: An attempt was made to access a socket in a way forbidden by its access permissions.


DEBUG - السياق المدمج:
التقنية: Laravel - الوصف: إطار عمل PHP يشتهر بجمالية الكود وسهولة الاستخدام للمطورين. 
التقنية: Vue.js - الوصف: إطار عمل مرن وسهل التعلم لبناء واجهات المواقع التفاعلية. 
التقنية: Flask - الوصف: إطار عمل Python مصغر (Micro) يتميز بالبساطة والمرونة المطلقة. 
التقنية: React - الوصف: إطار عمل JavaScript لبناء واجهات المستخدم، مدعوم من شركة Meta. 
التقنية: Django - الوصف: إطار عمل Python قوي جداً لتطوير المواقع بسرعة وبأمان عالٍ.
DEBUG - السياق المدمج:
التقنية: Laravel - الوصف: إطار عمل PHP يشتهر بجمالية الكود وسهولة الاستخدام للمطورين. 
التقنية: Vue.js - الوصف: إطار عمل مرن وسهل التعلم لبناء واجهات المواقع التفاعلية. 
التقنية: Flask - الوصف: إطار عمل Python مصغر (Micro) يتميز بالبساطة والمرونة المطلقة. 
التقنية: React - الوصف: إطار عمل JavaScript لبناء واجهات المستخدم، مدعوم من شركة Meta. 
التقنية: Django - الوصف: إطار عمل Python قوي جداً لتطوير المواقع بسرعة وبأمان عالٍ.
DEBUG - السياق المدمج:
التقنية: Laravel - الوصف: إطار عمل PHP يشتهر بجمالية الكود وسهولة الاستخد